In [1]:
import pandas as pd
import pickle
import dill

from trueskillthroughtime import Player, Game, History

In [2]:
folder = "C:/Users/jyoung/Projects/tennis_project/tennis_data/tennis_atp-master"

df_file = "processed_apt.csv"

dict_file = "trueskill_dict.pickle"

In [3]:
df = pd.read_csv(f'{folder}/{df_file}')

print(df.shape)
print("the last day in the df is:", df.tourney_date.max())

C:\Users\jyoung\AppData\Local\Temp\ipykernel_16904\739988522.py:1: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{folder}/{df_file}')


(572775, 146)
the last day in the df is: 2022-09-05


In [5]:
# TrueSkill Through Time to working its magic

columns = zip(df.winner_name, df.loser_name)

composition = [[[winner], [loser]] for winner, loser in columns]

true_skill_history = History(composition=composition,
                             sigma=1.6,
                             gamma=0.035)

In [8]:
ts_dict = true_skill_history.learning_curves()

In [6]:
# processing the histories

true_skill_history.convergence(epsilon=0.01, iterations=10)

Iteration =  0 , step =  (3.9066443759232747, 1.2479212223977658)
Iteration =  1 , step =  (0.2717825815512125, 0.08589906548698023)
Iteration =  2 , step =  (0.16198727776129918, 0.01286095868173942)
Iteration =  3 , step =  (0.12428970333117428, 0.010692837054082194)
Iteration =  4 , step =  (0.10795997852418449, 0.009608456347292682)
Iteration =  5 , step =  (0.1007442210761682, 0.008990357157518059)
Iteration =  6 , step =  (0.09559442759339554, 0.008492412337119726)
Iteration =  7 , step =  (0.09029010514771585, 0.00804583476555809)
Iteration =  8 , step =  (0.08509228234697996, 0.007626255621207889)
Iteration =  9 , step =  (0.0801448092590542, 0.007225488289093418)
End


((0.0801448092590542, 0.007225488289093418), 10)

In [9]:
# # creating a dictionary to count how many times a player has appeared in the dataset
match_count_dict = {name : 0 for name in ts_dict.keys()}

# # extracting the dictionary containing each player's sequence of TrueSkill ratings
# ts_dict = true_skill_history.learning_curves()

ts_win_probs = []

for winner, loser in zip(df.winner_name, df.loser_name):
  
  
  try:
    winner_idx = match_count_dict[winner]
    loser_idx = match_count_dict[loser]

    winner_player = Player(ts_dict[winner][winner_idx][1])
    loser_player = Player(ts_dict[loser][loser_idx][1])
    result = Game([[winner_player], [loser_player]]).evidence

    ts_win_probs.append(result)
  
    match_count_dict[winner] += 1
    match_count_dict[loser] += 1

  except:
    ts_win_probs.append(0.5)


df['winner_ts_win_prob'] = ts_win_probs

In [10]:
# saving the ts dictionary

dict_file = "trueskill_dict.pickle"

with open(f'{folder}/{dict_file}', 'wb') as handle:
    dill.dump(ts_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
df.to_csv(f'{folder}/processed_apt_with_ts.csv', index=False)